# Editing the FitnessCalculatorConfiguration: A Step-by-Step Guide

To run this notebook, you need to have an existing FitnessCalculatorConfiguration on Nexus.

Please fill in the values to match the metadata of the FitnessCalculatorConfiguration you want to edit.

In [8]:
emodel = "L5_TPC"
etype = "cAC"
mtype = None
ttype = None
iteration_tag = "XXXX-XX-XX"
species = "mouse"
brain_region = "SSCX"

nexus_project = "" # specify the nexus project to use
nexus_organisation = "bbp"
nexus_endpoint = "prod"

# Advanced settings (only change if you know what you are doing)
forge_path = "./forge.yml"
forge_ontology_path = "./nsg.yml"

This next cell will ask for your Nexus token:

In [ ]:
from bluepyemodel.access_point.nexus import NexusAccessPoint

data_access_point = "nexus"
access_point = NexusAccessPoint(
    emodel=emodel,
    species=species,
    brain_region=brain_region,
    project=nexus_project,
    organisation="bbp",
    endpoint="https://bbp.epfl.ch/nexus/v1",
    forge_path=forge_path,
    forge_ontology_path=forge_ontology_path,
    etype=etype,
    mtype=mtype,
    ttype=ttype,
    iteration_tag=iteration_tag
)

In [ ]:
fitness_config = access_point.get_fitness_calculator_configuration()

print(fitness_config)

In [18]:
from bluepyemodel.evaluation.efeature_configuration import EFeatureConfiguration
from bluepyemodel.evaluation.protocol_configuration import ProtocolConfiguration

As an example, we add a new protocol called `Step_1000`

In [19]:
protocol_name = "Step_1000"

stimuli = [{
    "amp": None,
    "delay": 700,
    "duration": 4,
    "holding_current": None,
    "thresh_perc": 1000,
    "totduration": 1000
}]

recordings = [
    {
        "type": "CompRecording",
        "location": "soma",
        "name": f"{protocol_name}.soma.v",
        "variable": "v"
    },
]

protocol_config = ProtocolConfiguration(
    name=protocol_name,
    stimuli=stimuli,
    recordings=recordings,
    validation=False
)

fitness_config.protocols.append(protocol_config)

Next, we add the feature associated with the newly defined protocol.

In [20]:
efeature_def = EFeatureConfiguration(
    efel_feature_name="Spikecount",
    protocol_name=protocol_name,
    recording_name="soma.v",
    mean=1.0,
    std=0.1,
    efel_settings={"strict_stiminterval": False},
    threshold_efeature_std=None,
)

fitness_config.efeatures.append(efeature_def)

Finally, we save the edited FitnessCalculatorConfiguration back to Nexus

In [ ]:
access_point.store_fitness_calculator_configuration(fitness_config)

To verify that the optimization works with the new FCC, run the following command:

In [ ]:
!python pipeline.py --step=test_optimise --emodel={emodel} --etype={etype} --iteration_tag={iteration_tag} --ttype={ttype}